In [65]:
import numpy as np
from scipy.io import loadmat
from sklearn.model_selection import KFold, RepeatedKFold
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge, RidgeCV, Lasso
from sklearn.neural_network import MLPRegressor
from scipy.stats import pearsonr, sem
from scipy import spatial
from sklearn.metrics import mean_squared_error, r2_score

In [66]:
def pairwise_accuracy(actual, predicted):
    true = 0
    total = 0
    for i in range(0,len(actual)):
#         print(i)
        for j in range(i+1, len(actual)):
            total += 1

            s1 = actual[i]
            s2 = actual[j]
            b1 = predicted[i]
            b2 = predicted[j]

            result1 = spatial.distance.cosine(s1, b1)
            result2 = spatial.distance.cosine(s2, b2)
            result3 = spatial.distance.cosine(s1, b2)
            result4 = spatial.distance.cosine(s2, b1)

            if(result1 + result2 < result3 + result4):
                true += 1

    return(true/total)

In [67]:
def pearcorr(actual, predicted):
    corr = []
    for i in range(0, len(actual)):
        corr.append(np.corrcoef(actual[i],predicted[i])[0][1])
    return np.mean(corr)

In [69]:
kf = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)

In [70]:
def train(vectors, voxels):
    
    dataset_X = np.array(voxels.copy())
    
    dataset_Y = np.array(vectors.copy())

#     accuracies = []
#     accuracies01 = []
    
    actual = []
    predicted = []
    pairwise_2v2 = []
    final_corr = []
    mse = []
    r2 = []
    #rdm_feat = []

    cnt = 0
    for train_index, test_index in kf.split(dataset_X):

        X_train, X_test = dataset_X[train_index], dataset_X[test_index]
        y_train, y_test = dataset_Y[train_index], dataset_Y[test_index]
           
        model = Ridge(alpha=1.0)
        model.fit(X_train,y_train)
        
        

        y_pred = model.predict(X_test)
        
        pairwise_2v2.append(pairwise_accuracy(y_test,y_pred))
        final_corr.append(pearcorr(y_test,y_pred))
        cnt += 1
    return np.mean(pairwise_2v2), np.mean(final_corr)

In [87]:
data = np.load('pieman_data_nli.npy',allow_pickle=True)
data = data.item()

In [89]:
ROIS = ['eac_L', 'eac_R', 'aac_L', 'aac_R', 'pmc_L', 'pmc_R', 'tpoj_L', 'tpoj_R', 'LAN_L', 'LAN_R']

In [53]:
for roi in ROIS:
    # data_voxels = loadmat('datafile.mat')
    data_voxels = np.load('../../afni-nosmooth/pieman_'+roi+'_avg.npy')
    data_voxels = data_voxels[:,data['non_empty_tr_indices'],:]
    print(data_voxels.shape)
    print(roi)
    print()
    output = {}
    output['2v2'] = {}
    output['pear'] = {}
    output['2v2'] = []
    output['pear'] = []
    for i in np.arange(82):
        roi_fmri = data_voxels[i]
        d,c = train(roi_fmri,data['tr_features'])
        #print(np.round(d,3),np.round(c,3), np.round(m,3), np.round(r2,3))
        output['2v2'].append(np.round(d,3))
        output['pear'].append(np.round(c,3))
    print(np.mean(output['2v2']), np.mean(output['pear']))
    fn = './'+roi+'_srl_ridge.npy'
    np.save(fn,output)

(82, 259, 3115)
vis_L

0.808731707317073 0.15658536585365854
(82, 259, 3091)
vis_R

0.8034878048780487 0.1568658536585366


In [74]:
for roi in ROIS:
    # data_voxels = loadmat('datafile.mat')
    data_voxels = np.load('../../afni-nosmooth/pieman_'+roi+'_avg.npy')
    data_voxels = data_voxels[:,data['non_empty_tr_indices'],:]
    print(data_voxels.shape)
    print(roi)
    print()
    output = {}
    output['2v2'] = {}
    output['pear'] = {}
    output['2v2'] = []
    output['pear'] = []
    for i in np.arange(82):
        roi_fmri = data_voxels[i]
        d,c = train(roi_fmri,data['tr_features'])
        #print(np.round(d,3),np.round(c,3), np.round(m,3), np.round(r2,3))
        output['2v2'].append(np.round(d,3))
        output['pear'].append(np.round(c,3))
    print(np.mean(output['2v2']), np.mean(output['pear']))
    fn = './'+roi+'_paraphrase_ridge.npy'
    np.save(fn,output)

(82, 259, 1061)
LAN_L

0.8274024390243901 0.17547560975609758
(82, 259, 875)
LAN_R

0.8154146341463415 0.1648048780487805


In [90]:
for roi in ROIS:
    # data_voxels = loadmat('datafile.mat')
    data_voxels = np.load('../../afni-nosmooth/pieman_'+roi+'_avg.npy')
    data_voxels = data_voxels[:,data['non_empty_tr_indices'],:]
    print(data_voxels.shape)
    print(roi)
    print()
    output = {}
    output['2v2'] = {}
    output['pear'] = {}
    output['2v2'] = []
    output['pear'] = []
    for i in np.arange(82):
        roi_fmri = data_voxels[i]
        d,c = train(roi_fmri,data['tr_features'])
        #print(np.round(d,3),np.round(c,3), np.round(m,3), np.round(r2,3))
        output['2v2'].append(np.round(d,3))
        output['pear'].append(np.round(c,3))
    print(np.mean(output['2v2']), np.mean(output['pear']))
    fn = './'+roi+'_summary_dec_ridge.npy'
    np.save(fn,output)

(82, 259, 808)
eac_L

0.6254878048780487 0.03958536585365854
(82, 259, 638)
eac_R

0.6255243902439024 0.044280487804878046
(82, 259, 1420)
aac_L

0.7432560975609757 0.0959390243902439
(82, 259, 1493)
aac_R

0.7508048780487805 0.10118292682926827
(82, 259, 1198)
pmc_L

0.818292682926829 0.20891463414634143
(82, 259, 1204)
pmc_R

0.8221341463414633 0.21559756097560978
(82, 259, 847)
tpoj_L

0.7905121951219513 0.16051219512195122
(82, 259, 1188)
tpoj_R

0.7959634146341464 0.1571219512195122
(82, 259, 1061)
LAN_L

0.8028902439024389 0.1591829268292683
(82, 259, 875)
LAN_R

0.7910121951219512 0.15104878048780487


In [91]:
methods = ['coreference', 'ner', 'nli', 'sa', 'shallosyntax', 'srl', 'paraphrase', 'summary', 'wsd', 'qa', 'el']
ROIS = ['eac_L', 'eac_R', 'aac_L', 'aac_R', 'pmc_L', 'pmc_R', 'tpoj_L', 'tpoj_R', 'LAN_L', 'LAN_R','vis_L', 'vis_R']

In [92]:
pcc = {}
acc_2v2 = {}
all_pcc = {}
all_2v2 = {}
for i in methods:
    pcc[i] = []
    acc_2v2[i] = []
    all_pcc[i] = []
    all_2v2[i] = []
    for roi in ROIS:
        #print(i, roi)
        if (i in 'NER') & (roi in ['LAN_L', 'LAN_R']):
            a=np.load(roi+'_ner_ridge.npy', allow_pickle=True)
            a = a.item()
        else:
            a=np.load(roi+'_'+i+'_ridge.npy', allow_pickle=True)
            a = a.item()
        #print(a['pear'][0],a['pear'][12], a['pear'][13])
        if i in 'summary_dec':
            print(np.array(a['pear']).shape)
        pcc[i].append(np.mean(a['pear']))
        acc_2v2[i].append(np.mean(a['2v2']))
        all_pcc[i].append(a['pear'])
        all_2v2[i].append(a['2v2'])

(82,)
(82,)
(82,)
(82,)
(82,)
(82,)
(82,)
(82,)
(82,)
(82,)
(82,)
(82,)


In [11]:
for i in methods:
    print(i, np.round(pcc[i],3))

coreference [0.037 0.042 0.077 0.077 0.153 0.159 0.122 0.12  0.112 0.103 0.121 0.121]
ner [0.025 0.029 0.064 0.067 0.143 0.145 0.11  0.108 0.109 0.099 0.116 0.116]
nli [0.05  0.055 0.104 0.108 0.219 0.224 0.168 0.166 0.16  0.15  0.173 0.174]
sa [0.028 0.029 0.07  0.071 0.151 0.156 0.117 0.116 0.115 0.108 0.123 0.124]
shallosyntax [0.012 0.014 0.035 0.034 0.084 0.088 0.066 0.061 0.061 0.055 0.067 0.068]
srl [0.039 0.045 0.088 0.091 0.191 0.199 0.149 0.144 0.146 0.137 0.157 0.157]
paraphrase_dec [0.071 0.07  0.131 0.14  0.248 0.27  0.208 0.21  0.203 0.181 0.184 0.186]
summary_dec [0.065 0.066 0.124 0.134 0.235 0.253 0.194 0.19  0.189 0.173 0.17  0.17 ]
wsd_dec [0.034 0.04  0.082 0.086 0.184 0.193 0.139 0.138 0.138 0.13  0.152 0.152]
qa [0.029 0.036 0.071 0.07  0.149 0.154 0.118 0.115 0.113 0.106 0.119 0.12 ]
el [0.028 0.029 0.065 0.065 0.142 0.147 0.107 0.109 0.104 0.098 0.114 0.115]


In [12]:
for i in methods:
    print(i, np.round(acc_2v2[i],3))

coreference [0.617 0.617 0.708 0.706 0.756 0.761 0.739 0.745 0.735 0.723 0.76  0.756]
ner [0.587 0.591 0.678 0.688 0.744 0.743 0.723 0.727 0.73  0.717 0.749 0.746]
nli [0.648 0.653 0.77  0.775 0.842 0.842 0.815 0.821 0.812 0.798 0.839 0.836]
sa [0.594 0.592 0.69  0.695 0.753 0.757 0.732 0.738 0.741 0.73  0.76  0.757]
shallosyntax [0.55  0.55  0.607 0.606 0.655 0.658 0.642 0.637 0.641 0.631 0.656 0.655]
srl [0.621 0.625 0.725 0.73  0.801 0.806 0.779 0.779 0.786 0.774 0.809 0.803]
paraphrase_dec [0.68  0.677 0.81  0.823 0.878 0.888 0.86  0.875 0.873 0.837 0.846 0.842]
summary_dec [0.666 0.67  0.793 0.808 0.86  0.868 0.839 0.848 0.852 0.824 0.824 0.818]
wsd_dec [0.613 0.617 0.726 0.735 0.803 0.811 0.773 0.781 0.783 0.774 0.81  0.805]
qa [0.603 0.612 0.699 0.697 0.759 0.762 0.741 0.744 0.741 0.733 0.757 0.755]
el [0.594 0.59  0.683 0.689 0.748 0.749 0.722 0.732 0.727 0.719 0.752 0.75 ]
